Data Processing

In [1]:
# Data import and cleaning

# Merged data
# Yearly
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.dates as mdates
import os

%config InlineBackend.figure_format = 'retina'
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Goolge colab
from google.colab import drive
drive.mount('/content/drive')

fmli_data = {}
veq_data = {}
for i in ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19']:
    for j in range(1, 5):
        if j == 1:
            path = f'/content/drive/MyDrive/URAP_Prof Yang/Raw Data/intrvw{i}/intrvw{i}/fmli{i}{j}x.csv'
        else:
            path = f'/content/drive/MyDrive/URAP_Prof Yang/Raw Data/intrvw{i}/intrvw{i}/fmli{i}{j}.csv'

        fmli_data[f'fmli_{i}{j}'] = pd.read_csv(path)

for i in ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19']:
    path = f'/content/drive/MyDrive/URAP_Prof Yang/Raw Data/intrvw{i}/expn{i}/veq{i}.csv'
    veq_data[f'veq_{i}'] = pd.read_csv(path)



for i in ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19']:
    veq_data[f'veq_{i}'] = veq_data[f'veq_{i}'][(veq_data[f'veq_{i}']['QYEAR'] // 10) == (2000 + int(i))] # Convert it into year
    veq_data[f'veq_{i}']['CUID'] = veq_data[f'veq_{i}']['NEWID'] // 10 # Consumer units id
    veq_data[f'veq_{i}']['QUARTER'] = veq_data[f'veq_{i}']['QYEAR'] % 10 # Gives you each quarter value


for i in ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19']:
    for j in range(1, 5):
      fmli_data[f'fmli_{i}{j}']['QYEAR'] = f'20{i}{j}'

for i in ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19']:
        fmli_data[f'fmli_{i}{j}']['CUID'] = fmli_data[f'fmli_{i}{j}']['NEWID'] // 10

for i in ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19']:
    for j in range(1, 5):
      fmli_data[f'fmli_{i}{j}']['QYEAR'] = f'20{i}{j}'

for i in ['10','11','12','13','14','15','16','17','18','19']:
    fmli_data[f'fmli_{i}'] = pd.concat([fmli_data[f'fmli_{i}{1}'], fmli_data[f'fmli_{i}{2}'], fmli_data[f'fmli_{i}{3}'], fmli_data[f'fmli_{i}{4}']], ignore_index=True)

for i in ['10','11','12','13','14','15','16','17','18','19']:
    fmli_data[f'fmli_{i}']['CUID'] = fmli_data[f'fmli_{i}']['CUID'].astype(int)
    fmli_data[f'fmli_{i}']['QYEAR'] = fmli_data[f'fmli_{i}']['QYEAR'].astype(int)

for i in ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19']:
  veq_data[f'veq_{i}']['CUID'].astype(int)
  veq_data[f'veq_{i}']['QYEAR'].astype(int)

merge = {}

for i in ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19']:
  # Using left-join to merge all of the columns
  merge[f'{i}'] = pd.merge(fmli_data[f'fmli_{i}'][['CUID','QYEAR','STATE']],veq_data[f'veq_{i}'][['QYEAR','CUID','VOPSERVY','VOPMOA','VOPEXPX']],on=['CUID', 'QYEAR'], how='left')
  # VOPEXPX is the total cost of the maintenance or repair expense
  merge[f'{i}']['VOPEXPX'] = pd.to_numeric(merge[f'{i}']['VOPEXPX'], errors='coerce') # fill na
  merge[f'{i}']['VOPEXPX'].fillna(0, inplace=True)
  # VOPMOA is an indicator variable for the month in which the expense occurred
  merge[f'{i}']['VOPMOA'] = pd.to_numeric(merge[f'{i}']['VOPMOA'], errors='coerce') # fill na
  merge[f'{i}']['VOPMOA'].fillna(0, inplace=True)
  merge[f'{i}']['repair_instances'] = merge[f'{i}']['VOPMOA'].isin(range(1, 13)).astype(int)
  merge[f'{i}']['total_repair_expenses'] = merge[f'{i}']['repair_instances'] * merge[f'{i}']['VOPEXPX']

final_merge = pd.concat([merge['10'],merge['11'],merge['12'],merge['13'],merge['14'],merge['15'],merge['16'],merge['17'],merge['18'],merge['19']], ignore_index=True)
final_merge = final_merge.rename(columns={'QYEAR':'YEAR','VOPEXPX': 'EXPENDITURE','VOPMOA' : "REPAIR_MONTH",'total_repair_expenses':'TOTAL_EXPENDITURE','VOPSERVY':'REPAIR_TYPE','repair_instances':'REPAIR_INSTANCES'})
final_merge.dropna(subset=["STATE"], inplace=True)

# Mapping csv by states
fips_to_state = {
    1: "Alabama",
    2: "Alaska",
    4: "Arizona",
    5: "Arkansas",
    6: "California",
    8: "Colorado",
    9: "Connecticut",
    10: "Delaware",
    11: "District of Columbia",
    12: "Florida",
    13: "Georgia",
    15: "Hawaii",
    16: "Idaho",
    17: "Illinois",
    18: "Indiana",
    19: "Iowa",
    20: "Kansas",
    21: "Kentucky",
    22: "Louisiana",
    23: "Maine",
    24: "Maryland",
    25: "Massachusetts",
    26: "Michigan",
    27: "Minnesota",
    28: "Mississippi",
    29: "Missouri",
    30: "Montana",
    31: "Nebraska",
    32: "Nevada",
    33: "New Hampshire",
    34: "New Jersey",
    35: "New Mexico",
    36: "New York",
    37: "North Carolina",
    38: "North Dakota",
    39: "Ohio",
    40: "Oklahoma",
    41: "Oregon",
    42: "Pennsylvania",
    44: "Rhode Island",
    45: "South Carolina",
    46: "South Dakota",
    47: "Tennessee",
    48: "Texas",
    49: "Utah",
    50: "Vermont",
    51: "Virginia",
    53: "Washington",
    54: "West Virginia",
    55: "Wisconsin",
    56: "Wyoming",
}

final_merge['STATE_NAME'] = final_merge['STATE'].map(fips_to_state)
final_merge = final_merge.reindex(columns=['CUID','YEAR','STATE','STATE_NAME','REPAIR_TYPE','REPAIR_MONTH','EXPENDITURE',	'REPAIR_INSTANCES','TOTAL_EXPENDITURE'])


# Repair_type distribution by type and number of repair instances of each household
plt.hist(final_merge['REPAIR_TYPE'])
# Distribution of repair instances of each household
final_merge['REPAIR_INSTANCES'].value_counts()

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/URAP_Prof Yang/Raw Data/intrvw10/intrvw10/fmli101x.csv'

# Machine Learning

In [ ]:
# Assign as a dataframe
aggregations = {
    'YEAR' : 'first',
    'CUID' : 'first',
    'STATE': 'first',
    'REPAIR_INSTANCES': 'sum',
    'REPAIR_TYPE' : 'first',
    'TOTAL_EXPENDITURE': 'sum'
}
cleaned_data = final_merge.groupby(['YEAR', 'CUID']).agg(aggregations)
cleaned_data['YEAR'] = cleaned_data['YEAR'] // 10

Logistic Regression with cross-validation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss
from sklearn.linear_model import LogisticRegression

cleaned_data.dropna(subset = 'REPAIR_TYPE',inplace = True)

# tf: Outcome Variable
# df: Controls(Types of Repair, Total expenditure)
tf = cleaned_data['REPAIR_INSTANCES']
df = cleaned_data[['REPAIR_TYPE','TOTAL_EXPENDITURE']]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss
from sklearn.linear_model import LogisticRegression

# With and Without intercept logistic regression and hamming loss error

df_train, df_test, tf_train, tf_test = train_test_split(df, tf, test_size=0.20)

clf_int = LogisticRegression(fit_intercept=True).fit(df_train, tf_train.values.ravel())
print('Hamming loss with intercept:', hamming_loss(tf_test, clf_int.predict(df_test)))

clf_no_int = LogisticRegression(fit_intercept=False).fit(df_train, tf_train.values.ravel())
print('Hamming loss with no intercept:', hamming_loss(tf_test, clf_no_int.predict(df_test)))

Hamming loss with intercept: 0.4858464064998487
Hamming loss with no intercept: 0.48601927481740786


10
x


KeyError: '[ 11570  11571  11572 ... 115690 115691 115692] not in index'